# Data Preperation

### Reading data into dataframes

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.io import mmread

# read the data
# frequency matrix. Format: 2-D Array[i][j] -> "freq of word i in article j". Shape: (9635, 2225). 1-based indexing.
freqMatrix = (mmread('./dataset/bbc.mtx'))
freqMatrix = freqMatrix.todense()
freqMatrixdf = pd.DataFrame(freqMatrix, range(1, freqMatrix.shape[0]+1), range(1, freqMatrix.shape[1]+1))

# article # to label. Format: Array[i] -> [cluster label of article i]. Shape: (2225, 1). 0-based indexing.
documentLabeldf = pd.read_csv('./dataset/bbc.classes', skiprows=4, sep=' ', header=None)
documentLabeldf.columns = ['article', 'label']
documentLabeldf = documentLabeldf.drop('article', axis=1)

# word # to word. Format: Array[i] -> [actual word at word i]. Shape: (9635, 1). 0-based indexing.
wordDictdf = pd.read_csv('./dataset/bbc.terms', header=None)
wordDictdf.columns = ['WORD']

# article # to article. Format: Array[i] -> [actual article at article i]. Shape: (2225, 1). 0-based indexing.
articleDictdf = pd.read_csv('./dataset/bbc.docs', header=None)
articleDictdf.columns = ['ARTICLE']

### Creating training and testing datasets (Binary values)

In [34]:
# type: numpy.ndarray. Shape: (2225, 9635)
# tranposed for better numpy indexing. mapped to binary values
toZeroOne = np.vectorize(lambda x: 1 if x > 0 else 0)
X_data = toZeroOne(freqMatrixdf.values.T)
# type: numpy.ndarray. Shape: (2225, 1)
y_data = documentLabeldf.values

# type: numpy.ndarray. Shape: (9635, 1)
wordDict = wordDictdf.values
# type: numpy.ndarray. Shape: (2225, 1)
articleDict = articleDictdf.values

# split the data into train and test
(X_train, X_test, y_train, y_test) = train_test_split(X_data, y_data, test_size=0.2, random_state=40)

print("X_train.shape: ", X_train.shape)
print("y_train.shape: ", y_train.shape)
print("X_test.shape: ", X_test.shape)
print("y_test.shape: ", y_test.shape)

X_train.shape:  (1780, 9635)
y_train.shape:  (1780, 1)
X_test.shape:  (445, 9635)
y_test.shape:  (445, 1)


# Naive Bayes Classifier

### Training the Bayes Classifier

In [35]:
def find_optimal_params_NB(X, y):
    """ Compute parameters for Naive Bayes classifier.
    Args:
    - X (ndarray (Shape: (N, D))): A NxD matrix corresponding to the inputs.
    - y (ndarray (Shape: (N, 1))): A N-column vector corresponding to the outputs given the inputs.
    
    Output:
    - A (ndarray (Shape: (D, K))): A NxD matrix corresponding to the ratio of the number of 1's in the i'th feature to its corresponding class j.
    - b (ndarray (Shape: (K, 1))): A 1-column vector corresponding to the ratio of the number of samples in class j to the total number of samples.
    """

    assert X.shape[0] == y.shape[0]
    N = X.shape[0]
    D = X.shape[1]
    K = y.max() + 1

    # calculating A matrix
    A = np.zeros((D, K))
    for label in range(K):
        numElemsInLabel = np.count_nonzero(y == label)
        whichIdxWithLabel = np.asarray(y == label).nonzero()[0]
        for feature in range(D):
            numOnesInFeatureAndLabel = np.count_nonzero(X[whichIdxWithLabel, feature])
            A[feature, label] = (numOnesInFeatureAndLabel + 1) / (numElemsInLabel + 2) # Laplace smoothing

    # calculating b vector
    b = np.zeros((K, 1))
    for label in range(K):
        numElemsInLabel = np.count_nonzero(y == label)
        b[label, 0] = numElemsInLabel / N
    
    assert (A >= 1).sum() == 0 # no feature has greater than or equal to 100% probability of being in a class
    assert (A <= 0).sum() == 0 # no feature has less than or equal to 0% probability of being in a class
    assert A.shape == (D, K)
    assert b.shape == (K, 1)

    return A, b

In [36]:
(A, b) = find_optimal_params_NB(X_train, y_train)

### Applying the Bayes Classifier

In [38]:
def get_pred_NB(A, b, X):
    """ Return predicted Y.
    Args:
    - X (ndarray (Shape: (N, D))): A NxD matrix corresponding to the inputs.
    - A (ndarray (Shape: (D, K))): A NxD matrix corresponding to the ratio of the number of 1's in the i'th feature to its corresponding class j.
    - b (ndarray (Shape: (K, 1))): A 1-column vector corresponding to the ratio of the number of samples in class j to the total number of samples.
    
    Output:
    - y_pred (ndarray (Shape: (N, 1))): A N-column vector corresponding to the outputs given the inputs.
    """
    N = X.shape[0]
    K = A.shape[1]

    y_pred = np.zeros((N, 1))

    for index, featureVector in enumerate(X):
        P_arr = np.zeros((K, 1))
        a_j_arr = np.zeros((K, 1))
        for label in range(K):
            whichIdxWithOnes = np.asarray(featureVector == 1).nonzero()[0]
            whichIdxWithZeroes = np.asarray(featureVector == 0).nonzero()[0]
            sumOfLnProbsOnes = np.log(A[whichIdxWithOnes, label]).sum()
            sumOfLnProbsZeroes = np.log(1-A[whichIdxWithZeroes, label]).sum()

            a_j = sumOfLnProbsOnes + sumOfLnProbsZeroes + np.log(b[label, 0])
            a_j_arr[label, 0] = a_j
        
        P_max = np.argmax(a_j_arr)
        y_pred[index, 0] = P_max
    
    return y_pred

### Evaluating the Bayes Classifier

In [39]:
# Training set accuracy
y_pred = get_pred_NB(A, b, X_train)
total_training = y_train.shape[0]
correct_training = np.count_nonzero(y_pred == y_train)
print(str.format("Training set accuracy: {:.2f}%", correct_training/total_training * 100))

# Testing set accuracy
y_pred = get_pred_NB(A, b, X_test)
total_testing = y_test.shape[0]
correct_testing = np.count_nonzero(y_pred == y_test)
print(str.format("Testing set accuracy: {:.2f}%", correct_testing/total_testing * 100))

Training set accuracy: 98.65%
Testing set accuracy: 95.96%


### Converting back to frequency values

In [42]:
X_data = freqMatrixdf.values.T

# split the data into train and test again
(X_train, X_test, y_train, y_test) = train_test_split(X_data, y_data, test_size=0.2, random_state=40)

print("X_train.shape: ", X_train.shape)
print("y_train.shape: ", y_train.shape)
print("X_test.shape: ", X_test.shape)
print("y_test.shape: ", y_test.shape)

X_train.shape:  (1780, 9635)
y_train.shape:  (1780, 1)
X_test.shape:  (445, 9635)
y_test.shape:  (445, 1)


# Gaussian Class Conditional Classifier

### Training the Gaussian Class Conditional Classifier

In [46]:
def calculate_Gaussian(x, mean, cov_mat_inv):
    """ Calculate Gaussian distribution.
    Args:
    - x (ndarray (Shape: (D, 1))): A D-column vector corresponding to the input.
    - mean (ndarray (Shape: (D, 1))): A D-column vector corresponding to the mean of the D features.
    - cov_mat_inv (ndarray (Shape: (D, D))): A DxD matrix corresponding to the inverse covariance matrix of the D features.
    
    Output:
    - Probability of data point x given mean and covariance.
    """
    x_minus_mean = x - mean
    exponent = -0.5 * (x_minus_mean.T @ cov_mat_inv @ x_minus_mean)

    return exponent.item() # without constant term or exponent

def find_optimal_params_GCC(X, y, bias_=0.0001):
    """ Compute parameters for Gaussian Class Conditional classifier.
    Args:
    - X (ndarray (Shape: (N, D))): A NxD matrix corresponding to the inputs.
    - y (ndarray (Shape: (N, 1))): A N-column vector corresponding to the outputs given the inputs.
    
    Output:
    - means (ndarray (Shape: (D, K))): A D-column vector corresponding to the means of the D features for each class.
    - cov_mats (ndarray (Shape: (D, D, K))): A DxD matrix corresponding to the covariance matrix of the D features for each class.
    - priors (ndarray (Shape: (K, 1))): A 1-column vector corresponding to the prior probability of each class.
    - features_included (ndarray (Shape: (D, K))): A D-column vector corresponding to the features included in the class.
    """
    assert X.shape[0] == y.shape[0]
    N = X.shape[0]
    D = X.shape[1]
    K = np.amax(y, axis=0).item() + 1

    # Initializing the parameters
    means = []
    cov_mat_invs = []
    priors = np.zeros((K, 1))
    features_included = np.zeros((D, K))

    for label in range(K):
        X_with_label = X[y[:, 0] == label]
        numElemsInLabel = X_with_label.shape[0]

        # Reducing the number of features by removing features with variance of 0
        variance_of_features = np.var(X_with_label, axis=0)
        features_included[:,label] = np.where(variance_of_features > 0, 1, 0)

        # Computing the mean and covariance matrix for each class
        means.append(np.mean(X_with_label[:, features_included[:, label] == 1], axis=0).reshape(-1,1))
        
        # Adding a small bias to the covariance matrix to avoid singular matrix
        cov_mat = np.cov(X_with_label[:, features_included[:, label] == 1], rowvar=False)
        bias = bias_ * np.eye(cov_mat.shape[0])
        cov_mat = cov_mat + bias
        cov_mat_inv = np.linalg.inv(cov_mat)
        cov_mat_invs.append(cov_mat_inv)

        # Computing the prior probability for each class
        priors[label, 0] = numElemsInLabel / N

    return means, cov_mat_invs, priors, features_included

### Applying the Gaussian Class Conditional Classifier

In [47]:
def get_pred_GCC(means, cov_mat_invs, priors, features_included, X):
    """ Compute prediction for Gaussian Class Conditional classifier.
    Args:
    - means (ndarray (Shape: (D, K))): A D-column vector corresponding to the means of the D features for each class.
    - cov_mat_invs (ndarray (Shape: (D, D, K))): A DxD matrix corresponding to the inverse covariance matrix of the D features for each class.
    - priors (ndarray (Shape: (K, 1))): A 1-column vector corresponding to the prior probability of each class.
    - features_included (ndarray (Shape: (D, K))): A D-column vector corresponding to the features included in the class.
    - X (ndarray (Shape: (N, D))): A NxD matrix corresponding to the inputs.
    
    Output:
    - y_pred (ndarray (Shape: (N, 1))): A N-column vector corresponding to the predicted outputs given the inputs.
    """
    N = X.shape[0]
    D = X.shape[1]
    K = features_included.shape[1]
    y_pred = np.zeros((N, 1))

    for i in range(N):
        max_prob = np.NINF
        for j in range(K):
            x_zero_features_removed = X[i, :].reshape(D, 1)[features_included[:, j] == 1, 0].reshape(-1, 1)
            
            prob = (calculate_Gaussian(x_zero_features_removed, means[j], cov_mat_invs[j]) * priors[j, 0]).item()
            if prob > max_prob:
                max_prob = prob
                y_pred[i, 0] = j

    return y_pred

In [48]:
# get the optimal parameters
(means, cov_mat_invs, priors, features_included) = find_optimal_params_GCC(X_train, y_train, bias_=0.00000001)

### Evaluating the Gaussian Class Conditional Classifier

In [49]:
# Training set accuracy
y_pred = get_pred_GCC(means, cov_mat_invs, priors, features_included, X_train)
total_training = y_train.shape[0]
correct_training = np.count_nonzero(y_pred == y_train)
print(str.format("Training set accuracy: {:.2f}%", correct_training/total_training * 100))

# Testing set accuracy
y_pred = get_pred_GCC(means, cov_mat_invs, priors, features_included, X_test)
total_testing = y_test.shape[0]
correct_testing = np.count_nonzero(y_pred == y_test)
print(str.format("Testing set accuracy: {:.2f}%", correct_testing/total_testing * 100))

Training set accuracy: 100.00%
Testing set accuracy: 69.44%


### Results

The results for the Gaussian Class Conditional Classifier is as follows:

Training set accuracy: 100.00% <br>
Testing set accuracy: 69.44%

# K-NN Classifier

### Applying the K-NN Classifier

In [50]:
def kNN_Classify(X_train, y_train, X, k):
    """ Classifier using K-NN.
    Args:
    - X_train (ndarray (Shape: (N, D))): A NxD matrix corresponding to the training inputs.
    - X (ndarray (Shape: (M, D))): A MxD matrix corresponding to the testing inputs.
    - y_train (ndarray (Shape: (N, 1))): A N-column vector corresponding to the outputs given the inputs.
    - k (int): Number of nearest neighbors to consider.
    
    Output:
    - y_pred (ndarray (Shape: (M, 1))): A M-column vector corresponding to the outputs given the inputs.
    """
    N = X_train.shape[0]
    M = X.shape[0]

    y_pred = np.zeros((M, 1))

    for index, featureVector in enumerate(X):
        dist_arr = np.zeros((N, 1))

        for i in range(N):
            dist = np.linalg.norm(featureVector - X_train[i])
            dist_arr[i, 0] = dist

        k_smallest_indexes = np.argpartition(dist_arr, k, axis=0)[:k].flatten()
        labels_k_smallest = y_train[k_smallest_indexes].flatten()
        label_counts = np.bincount(labels_k_smallest)

        y_pred[index, 0] = np.argmax(label_counts)

    return y_pred

### Evaluating the K-NN Classifier

In [52]:
# test the classifier
testing_arr = [1, 3, 6]
for k in testing_arr:
    # training set accuracy
    y_pred = kNN_Classify(X_train, y_train, X_train, k)
    total_testing = y_train.shape[0]
    correct_testing = np.count_nonzero(y_pred == y_train)
    print(str.format("Training set accuracy: {:.2f}% for k = {}", correct_testing/total_testing * 100, k))

    # testing set accuracy
    y_pred = kNN_Classify(X_train, y_train, X_test, k)
    total_testing = y_test.shape[0]
    correct_testing = np.count_nonzero(y_pred == y_test)
    print(str.format("Testing set accuracy: {:.2f}% for k = {}", correct_testing/total_testing * 100, k))


Training set accuracy: 100.00% for k = 1
Testing set accuracy: 79.33% for k = 1
Training set accuracy: 86.85% for k = 3
Testing set accuracy: 70.11% for k = 3
Training set accuracy: 72.02% for k = 6
Testing set accuracy: 64.72% for k = 6


### Results

The result for the K-NN classifier is as follows:

Training set:
<ul>
    <li> Training set accuracy: 100.00% for k = 1
    <li> Training set accuracy: 86.85% for k = 3
    <li> Training set accuracy: 72.02% for k = 6
</ul>

Testing set:
<ul>
    <li> Testing set accuracy: 79.33% for k = 1
    <li> Testing set accuracy: 70.11% for k = 3
    <li> Testing set accuracy: 64.72% for k = 6
</ul>